In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgbm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.plots import plot_convergence
import json
import string

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes,linear_model
from sklearn.ensemble import RandomForestClassifier

import re
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
from keras import losses
from keras import metrics
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from keras.layers import Embedding, Flatten, Dense, GRU
from keras.utils import to_categorical
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from collections import Counter
from pathlib import Path
import os
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import unicodedata
import html
stop_words = stopwords.words('english')


import warnings
import numpy as np 
import pandas as pd 
import os
import re
import nltk
from nltk.corpus import abc
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.stem.snowball import *
from nltk.util import ngrams
import string
import spacy
from spacy import displacy
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from wordcloud import WordCloud, STOPWORDS
import gensim
from gensim.models.word2vec import Text8Corpus
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
#from spacytextblob.spacytextblob import SpacyTextBlob
from IPython.core.display import HTML
import seaborn as sns
import matplotlib.pyplot as plt
import html
%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

rand_seed = 13579
np.random.seed(rand_seed)

sns.set(style="darkgrid", context="notebook")

In [ ]:
train_df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test_df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

In [ ]:
#Sample of the data
idx_1 = train_df.query('target==0').sample(n=10000, random_state=42).index
idx_0 = train_df.query('target!=0').sample(n=8000, random_state=42).index
train_df = train_df.loc[list(idx_1) + list(idx_0), :]

In [ ]:
train_df.head()

In [ ]:
x_train = train_df['comment_text'].fillna('').values

#if true, y_train[i] =1, if false, it is = 0
y_train = np.where(train_df['target'] >= 0.5, 1, 0)

y_aux_train = train_df[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]

x_train = train_df['comment_text'].fillna('').values

x_test = test_df['comment_text'].fillna('').values


In [ ]:
vocab_sz = 10000
tok = Tokenizer(num_words=vocab_sz, oov_token='UNK')

In [ ]:
tok.fit_on_texts(list(x_train) + list(x_test))
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test)

In [ ]:
#x_train

In [ ]:
x_train = pad_sequences(x_train, maxlen=220)
x_test = pad_sequences(x_test, maxlen=220)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
embedding_index = {}

In [ ]:
f = open(os.path.join('./glove.6B.200d.txt'), encoding='utf8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()

In [ ]:
word_index = tok.word_index

In [ ]:
maxlen = 220
max_words = 10000
embedding_dim = 200
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
x_train_text, x_val_text, y_train_text, y_val_text = train_test_split(x_train, y_train, test_size=0.4, random_state=41)

In [ ]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(GRU(64, return_sequences=True))
model.add(GRU(64))
model.add(Dense(1, activation="sigmoid"))
model.summary()


In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
from keras.optimizers import Adam

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(clipnorm=0.1),
        metrics=['accuracy'])


history = model.fit(x_train,
                    y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val_text, y_val_text))
history_dict = history.history
history_dict.keys()

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import LearningRateScheduler



words = Input(shape=(maxlen,))
x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words) #Finds word embeddings for each word
x = SpatialDropout1D(0.3)(x) #This version performs the same function as Dropout, however it drops entire 1D feature maps instead of individual elements
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
hidden = concatenate([
    GlobalMaxPooling1D()(x), 
    GlobalAveragePooling1D()(x),#layer returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model to handle input 
    #of variable length in the simplest way possible.
])
hidden = add([hidden, Dense(4*128, activation='relu')(hidden)]) #This fixed-length output vector is piped through a fully-connected (Dense) layer with x hidden units.
hidden = add([hidden, Dense(4*128, activation='relu')(hidden)])
result = Dense(1, activation='sigmoid')(hidden)
aux_result = Dense(6, activation='sigmoid')(hidden)
model = Model(inputs=words, outputs=[result, aux_result])
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
for global_epoch in range(4):
    model.fit(
        x_train,
        [y_train, y_aux_train],
        batch_size=512,
        epochs=1,
        verbose=1,
        callbacks=[
            LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch), verbose = 1)
        ]
    )